In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [ ]:
df = pd.read_csv("bodyfat.csv")         # Importing El- nino dataset
df.head(50)

In [ ]:
data = df.values
data[:5,:]

$\text{Define data augmentation function.}$

In [ ]:
def augment_data(original_data, num_samples=20000, num_neighbors=6):
    augmented_data = []
    nn = NearestNeighbors(n_neighbors=num_neighbors)
    nn.fit(original_data)

    for _ in range(num_samples):
        # Randomly select a point from the original dataset
        random_index = np.random.randint(0, len(original_data))
        random_point = original_data[random_index]

        # Find the indices of the nearest neighbors
        d, indices = nn.kneighbors(random_point.reshape(1, -1))  # d is the distance between the random point and 5 niegh
        neighbors = original_data[indices[0]]

        # Average the values of the random point and its neighbors
        new_point = np.mean(neighbors, axis=0)
        augmented_data.append(new_point)

    return np.array(augmented_data)

In [ ]:
# Augment the data
data = augment_data(data)

# Now augmented_data contains the augmented dataset with 20,000 data points
print("Shape of augmented data:", data.shape)

In [ ]:
np.mean(data, axis = 0)

In [ ]:
def making_matrix(data, size):
    np.random.shuffle(data)

    num_rows = data.shape[0]

    # Randomly select 100 rows for s1
    s1_indices = np.random.choice(num_rows, size=size, replace=False)
    s1_matrix = data[s1_indices]

    # Remove the selected rows from data to create s1_complement
    s1_complement_matrix = np.delete(data, s1_indices, axis=0)

    # Randomly select 100 rows from s1_complement for s2
    s2_indices = np.random.choice(s1_complement_matrix.shape[0], size=size, replace=False)
    s2_matrix = s1_complement_matrix[s2_indices]

    # Remove the selected rows from data to create s1_complement
    s2_complement_matrix = np.delete(s1_complement_matrix, s2_indices, axis=0)

    # Randomly select 100 rows from s1_complement for s2
    sprime_indices = np.random.choice(s2_complement_matrix.shape[0], size=size, replace=False)
    sprime_matrix = s2_complement_matrix[sprime_indices]

    return s1_matrix, s2_matrix, sprime_matrix

In [ ]:
a=np.array([[1,2,3],[4,5,6],[7,8,9]])
b = np.delete(a,[1,2],axis=0)
a

In [ ]:
b

In [ ]:
s1_matrix , s2_matrix, sprime_matrix = making_matrix(data, 100)

print(s1_matrix.shape)
print(s2_matrix.shape)
print(sprime_matrix.shape)

len_s1= s1_matrix.shape[0]
len_s2= s2_matrix.shape[0]
len_sprime = sprime_matrix.shape[0]

len_s1, len_s2, len_sprime

In [ ]:
s1_matrix

In [ ]:
s2_matrix

In [ ]:
sprime_matrix

$\Huge{\text{Required Fuctions for Alogorithm}}$

# $ \textbf{Equation 5:} \\
\left\{\begin{array}{l}
p\left(x_{j} \mid \omega_{i}\right)=\frac{1}{(2 \pi)^{k / 2}\left|\Sigma_{x_{i}}\right|^{1 / 2}} \exp \left(-\frac{1}{2}\left(x_{j}-x_{i}\right)^{T} \Sigma_{x_{i}}^{-1}\left(x_{j}-x_{i}\right)\right) \\
p\left(x_{i} \mid \omega_{i}\right)=0, \quad i, j=1,2, \ldots\left|S_{1}\right|, i \neq j \\
\quad i=1,2, \ldots\left|S_{1}\right|
\end{array}\right.
$

In [ ]:
dim = 15        # Dimensionality of s1

def small_p(xi, xj, cov):

    cov_1 = cov

    det_cov = np.linalg.det(cov)
    epsilon = 1e-5
    det_cov = max(det_cov, epsilon)

    if det_cov <= 0.01:
        alpha = 1e-5
        cov_1 = cov_1 + alpha * np.identity(dim)

    a = 1 / (((2 * np.pi) ** (dim / 2)) * det_cov ** 0.5)
    b = -0.5 * ((xj - xi).T @ np.linalg.inv(cov_1) @ (xj - xi))

    # if b < 0:
    #     print('b value', b)
    b = max(b, -4)
    p = a * np.exp(b)

    return p

# $ \textbf{Equation 4:} \\
\begin{cases}P\left(\omega_{i} \mid x_{j}\right)=\frac{p\left(x_{j} \mid \omega_{i}\right)}{\sum_{t=1}^{\left|S_{1}\right|} p\left(x_{j} \mid \omega_{t}\right)} & i, j=1,2, \ldots\left|S_{1}\right|, i \neq j \\ P\left(\omega_{i} \mid x_{i}\right)=0 & i=1,2, \ldots\left|S_{1}\right|\end{cases}
$

In [ ]:
def capital_p(i, j, cov_mat):
    summation = 0
    xi = s1_matrix[i, :].reshape(-1, 1)
    xj = s1_matrix[j, :].reshape(-1, 1)

    for k in range(len_s1):
        if k != j:
            xk = s1_matrix[k, :].reshape(-1, 1)
            summation += small_p(xk,xj, cov_mat[k])

    # Avoid division by zero
    summation = np.clip(summation,1e-4,np.inf)
    P = small_p(xi,xj,cov_mat[i]) / summation

    return  P

# $ \textbf{Equation 3:} \\
\Sigma_{x_{i}}=\frac{\sum_{x_{j} \in S_{1}} P\left(\omega_{i} \mid x_{j}\right)\left(x_{j}-x_{i}\right)\left(x_{j}-x_{i}\right)^{T}}{\sum_{x_{j} \in S_{1}} P\left(\omega_{i} \mid x_{j}\right)}
$

In [ ]:
def cov_xi(i,cov_mat):
    denominator = 0
    numerator = 0

    xi = s1_matrix[i, :].reshape(-1, 1)
    for j in range(len_s1):
        if i!=j:
            xj = s1_matrix[j, :].reshape(-1, 1)

            P = capital_p(i, j, cov_mat)

            denominator += P
            numerator += P * ((xj - xi) @ (xj - xi).T)

    return numerator / denominator

# $ \text{Equation 2:}$
$ \ L=\sum_{x_{j} \in S_{1}} \log \left[\sum_{x_{i} \in S_{1} \wedge i \neq j} \frac{1}{\left|S_{1}\right|-1} G\left(\Sigma_{x_{i}}, x_{j}-x_{i}\right)\right]
$

In [ ]:
def L(cov_mat):
    L_new = 0
    for j in range(len_s1):
        xj = s1_matrix[j, :].reshape(-1, 1)
        summation = 0
        for i in range(len_s1):
            if i!=j:
                xi = s1_matrix[i, :].reshape(-1, 1)
                summation+=small_p(xi,xj,cov_mat[i])

        # summation = np.clip(summation,1e-5,np.inf)
        summation=summation/(len_s1-1)
        L_new+=np.log(summation)

    return L_new[0][0]

$\Huge{\text{Alogorithm 1}}$

In [ ]:
def Algorithm1(cov_mat):
    t = 0
    phi = 0.01
    L_prev = 1

    while t < 100:

        temp = []

        for i in range(len_s1):
            temp.append(cov_xi(i,cov_mat))

        cov_mat = temp.copy()

        for i in range(len_s1):
            if np.linalg.det(cov_mat[i]) <= 0:
                alpha = 1
                cov_mat[i] = cov_mat[i] + alpha * np.identity(dim)

        L_new = L(cov_mat)

        print(((L_new-L_prev)/L_prev))
        print('L_prev=',L_prev)
        print('L_new=',L_new)
        print('t=',t)
        print('-'*50)

        if abs((L_new-L_prev)/L_prev) < phi:
            break

        else:
            L_prev=L_new
            t+=1

    return cov_mat

In [ ]:
#initial guess
cov_mat_initial=[]
for i in range(len_s1):
    xi = s1_matrix[i, :].reshape(-1, 1)
    mat=0
    for j in range(len_s1):
        xj = s1_matrix[j, :].reshape(-1, 1)
        mat+=(xj-xi)@(xj-xi).T
    cov_mat_initial.append(mat/(len_s1-1))

In [ ]:
cov_mat_initial[0].shape

In [ ]:
cov_mat= Algorithm1(cov_mat_initial)

In [ ]:
cov_mat

$\Huge{\text{Alogorithm 2}}$

In [ ]:
def F(T):
    s = 0
    for i in range(len_s1):
        xi = s1_matrix[i, :].reshape(-1, 1)
        s += small_p(xi,T,cov_mat[i])
    f = np.log(s/len_s1)
    return f

def EstVar(estsize,beta):
    Est = []
    for t in range(1,estsize):
        np.random.seed(42)  # Setting the random seed for reproducibility
        R_indices = np.random.choice(s2_matrix.shape[0], size=len(s2_matrix), replace=True)
        R = s2_matrix[R_indices]
        F_R = []
        for i in range(len_s2):
            T = R[i, :].reshape(-1, 1)
            F_R.append(F(T))

        var_F_R = np.var(np.array(F_R))
        Est.append((len_s2/(len_s2-1))*var_F_R)

    V =  np.percentile(Est, estsize*(1 - beta))
    var_delta = (len_sprime + ((len_sprime**2) / len_s2))*V
    return var_delta

$\Huge{\text{Alogorithm 3}}$

In [ ]:
from scipy.stats import norm

def critical_value(p,stepsize):
    estsize = 100
    M = int(np.ceil((p/stepsize)-1))
    C = []
    for i in range(1, M+1):
        print('step number:',i,'/',M+1)
        alpha_i = i*stepsize
        beta_i = p-alpha_i
        var_delta = EstVar(estsize,beta_i)

        mean = 0

        std_dev = np.sqrt(var_delta)
        c = norm.ppf(alpha_i, loc = mean, scale = std_dev)
        C.append(c)

    C_max = max(C)
    print('C_max', C_max)

    return C_max

# $\textbf{Equation 7:}$
 $
\begin{aligned}
\delta= & \operatorname{LLH}\left(\mathscr{K}_{S_{1}}, S^{\prime}\right)-\frac{\left|S^{\prime}\right|}{\left|S_{2}\right|} \times \operatorname{LLH}\left(\mathscr{K}_{S_{1}}, S_{2}\right) \\
= & \log \left\{\prod_{y \in S^{\prime}} \mathscr{K}_{S_{1}}(y)\right\}-\frac{\left|S^{\prime}\right|}{\left|S_{2}\right|} \times \log \left\{\prod_{y \in S_{2}} \mathscr{K}_{S_{1}}(y)\right\} \\
= & \sum_{y \in S^{\prime}} \log \sum_{x \in S_{1}} \frac{1}{\left|S_{1}\right|} G\left(\Sigma_{x}, y-x\right) \\
& -\sum_{y \in S_{2}} \frac{\left|S^{\prime}\right|}{\left|S_{2}\right|} \times \log \left\{\sum_{x \in S_{1}} \frac{1}{\left|S_{1}\right|} G\left(\Sigma_{x}, y-x\right)\right\}
\end{aligned}
$

In [ ]:
def delta(sprime_matrix):

    a = 0

    for j in range(len_sprime):
        y = sprime_matrix[j, :].reshape(-1, 1)
        summation = 0

        for i in range(len_s1):
            x = s1_matrix[i, :].reshape(-1, 1)
            summation += small_p(x,y,cov_mat[i])

        summation=np.clip(summation,1e-4,np.inf)
        a += np.log(summation/len_s1)

    b = 0

    for j in range(len_s2):
        y = s2_matrix[j, :].reshape(-1, 1)
        summation = 0

        for i in range(len_s1):
            x = s1_matrix[i, :].reshape(-1, 1)
            summation += small_p(x,y,cov_mat[i])

        b += np.log(summation/len_s1)

    b = b*len_sprime/len_s2

    delta = a - b
    print('delta', delta)

    return delta

In [ ]:
sprime_matrix.shape

$\Huge{\text{Change Detection Decision}}$

In [ ]:
p = 0.08
stepsize = 0.002

Delta = delta()

C_max = critical_value(p, stepsize)

if Delta < C_max:
    print('THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S ')

else:
    print('THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S ')

$\Huge{\text{Making a Hypothetical Data to Check Algorithm}}$

In [ ]:
len_sprime

In [ ]:
sprime_matrix

In [ ]:
A = sprime_matrix.copy()

for j in range(dim):
    for i in range(len_sprime):

        min_values = 4*np.min(sprime_matrix, axis=0)[j]
        max_values = 2*np.max(sprime_matrix, axis=0)[j]
        A[i,j]=np.random.uniform(min_values,max_values)


sprime_matrix = A.copy()

In [ ]:
sprime_matrix.shape

In [ ]:
sprime_matrix

$\Huge{\text{Change Detection for Hypothetical Data}}$ \\
$\text{}$

In [ ]:
#Change detection
p=0.08
stepsize=0.002

Delta = delta(sprime_matrix)
C_max= critical_value(p,stepsize)

if Delta<C_max:
    print('THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S ')

else:
    print('THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S ')

$\Huge{\text{Two way test}}$
$\text{}$

In [ ]:
s1_matrix , s2_matrix, sprime_matrix = making_matrix(data, 100)

S = sprime_matrix.copy()

sprime_matrix = np.concatenate((s1_matrix, s2_matrix), axis=0)


# Shuffle the indices of the matrix
indices = np.random.permutation(S.shape[0])

# Divide the shuffled indices into two equal parts
split_index = len(indices) // 2
indices_1 = indices[:split_index]
indices_2 = indices[split_index:]

# Create two matrices using the selected indices
s1_matrix = S[indices_1]
s2_matrix = S[indices_2]


print(s1_matrix.shape)
print(s2_matrix.shape)
print(sprime_matrix.shape)

len_s1= s1_matrix.shape[0]
len_s2= s2_matrix.shape[0]
len_sprime = sprime_matrix.shape[0]

len_s1, len_s2, len_sprime

In [ ]:
#initial guess
cov_mat_initial=[]
for i in range(len_s1):
    xi = s1_matrix[i, :].reshape(-1, 1)
    mat=0
    for j in range(len_s1):
        xj = s1_matrix[j, :].reshape(-1, 1)
        mat+=(xj-xi)@(xj-xi).T
    cov_mat_initial.append(mat/(len_s1-1))

In [ ]:
#ALGORITHM 1

cov_mat=Algorithm1(cov_mat_initial)


In [ ]:
cov_mat

In [ ]:
#Change detection
p=0.08/2
stepsize=0.002

Delta = delta()
C_max= critical_value(p,stepsize)

if Delta<C_max:
    print('THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S ')

else:
    print('THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S ')

$\Huge{\text{Experiment 1}}$
$\text{}$

In [ ]:
fp=0

for i in range(20):
    print('itr=',i)
    s1_matrix , s2_matrix, sprime_matrix = making_matrix(data, 100)

    len_s1 = s1_matrix.shape[0]
    len_s2 = s2_matrix.shape[0]
    len_sprime = sprime_matrix.shape[0]

    #initial guess
    cov_mat_initial = []
    for i in range(len_s1):
        xi = s1_matrix[i, :].reshape(-1, 1)
        mat = 0
        for j in range(len_s1):
            xj = s1_matrix[j, :].reshape(-1, 1)
            mat += (xj-xi)@(xj-xi).T
        cov_mat_initial.append(mat/(len_s1-1))

    cov_mat = Algorithm1(cov_mat_initial)

    #Change detection
    p = 0.08/2
    stepsize = 0.002

    Delta = delta()
    C_max = critical_value(p,stepsize)

    if Delta < C_max:
        fp += 1
        print('THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S ')

    else:
        print('THERE IS NO CHANGE. DISTRIBUTION OF S PRIME IS SAME AS DISTRIBUTION OF S ')

        s1_matrix , s2_matrix, sprime_matrix = making_matrix(data, 100)

        S = sprime_matrix.copy()

        sprime_matrix = np.concatenate((s1_matrix, s2_matrix), axis=0)


        # Shuffle the indices of the matrix
        indices = np.random.permutation(S.shape[0])

        # Divide the shuffled indices into two equal parts
        split_index = len(indices) // 2
        indices_1 = indices[:split_index]
        indices_2 = indices[split_index:]

        # Create two matrices using the selected indices
        s1_matrix = S[indices_1]
        s2_matrix = S[indices_2]

        len_s1= s1_matrix.shape[0]
        len_s2= s2_matrix.shape[0]
        len_sprime = sprime_matrix.shape[0]

        #initial guess
        cov_mat_initial=[]
        for i in range(len_s1):
            xi = s1_matrix[i, :].reshape(-1, 1)
            mat=0
            for j in range(len_s1):
                xj = s1_matrix[j, :].reshape(-1, 1)
                mat+=(xj-xi)@(xj-xi).T
            cov_mat_initial.append(mat/(len_s1-1))

        cov_mat = Algorithm1(cov_mat_initial)

        #Change detection

        p=0.08/2
        stepsize=0.002

        Delta = delta()
        C_max= critical_value(p,stepsize)

        if Delta<C_max:
            fp+=1
            print('THERE IS CHANGE. DISTRIBUTION OF S PRIME IS DIFFERENT THAN DISTRIBUTION OF S')

        print('fp:',fp)
        print('')

print('FALSE POSITIVE RATE:', fp/20)